# seaMass-1D demo
This notebook illustrates how to load seaMass input/output in 1D. Firstly, we need to import the relevant modules...

In [1]:
using Plots; plotly(size=(980,350))
using Interact
import SeaMass

### 1. Load a spectrum from an mzMLb file

In [5]:
mzmlSpectrum = SeaMass.MzmlSpectrum("../data/P02U_Swath_1_1D.mzMLb", 1)

plot(mzmlSpectrum.mzs, mzmlSpectrum.intensities, label="mzML input")
plot!(xlabel = "m/z (Th)", ylabel = "intensity")

### 2. Load a spectrum from an SMI file

In [6]:
smiSpectrum = SeaMass.SmiSpectrum("../data/P02U_Swath_1_1D.pos_617.55.smi", 1)

plot(mzmlSpectrum.mzs, mzmlSpectrum.intensities * smiSpectrum.exposure, label="mzML input")
plot!(smiSpectrum.binEdges, vcat(smiSpectrum.binCounts, 0.0), line=:steppost, label="SMI input")
plot!(xlabel = "m/z (Th)", ylabel = "ion count")

### 3. Load seaMass output from SMV and SMO files
We cannot process the B-spline basis functions returned in the SMV file yet, but we can read the residuals and subtract this from the SMI input bin counts in order to get the seaMass predicted bin counts. In order to compare the two however, we need to make the y axis the ion density.

In [ ]:
smiBinWidths = smiSpectrum.binEdges[2:end] - smiSpectrum.binEdges[1:end-1]

smvSpectrum = SeaMass.SmvSpectrum("../data/P02U_Swath_1_1D.pos_617.55.smv", 1)
smvBinCounts = smiSpectrum.binCounts - smvSpectrum.residualBinCounts

smoSpectrum = SeaMass.SmoSpectrum("../data/P02U_Swath_1_1D.pos_617.55.smo", 1)
binnedSmoSpectrum = SeaMass.BinnedSmoSpectrum(smoSpectrum)
sampledSmoSpectrum = SeaMass.MzmlSpectrum(smoSpectrum, 100000)

plot(smiSpectrum.binEdges, vcat(smiSpectrum.binCounts  ./ smiBinWidths, 0.0), line=:steppost, label="SMI input")
plot!(smiSpectrum.binEdges, vcat(smvBinCounts ./ smiBinWidths, 0.0), line=:steppost, label="seaMass prediction")
plot!(binnedSmoSpectrum.binEdges, vcat(binnedSmoSpectrum.binCounts / binnedSmoSpectrum.binWidth, 0.0),
      line=:steppost, label="binned reconstruction")
plot!(sampledSmoSpectrum.mzs, sampledSmoSpectrum.intensities, label="sampled reconstruction")
plot!(xlabel = "m/z (Th)", ylabel = "ion density (counts per Th)")
plot!(ylims = (0., 35000.)) 

### 5. Animations!

In [ ]:
smvFilenames = filter!(r"^P02U_Swath_1_1D\.pos_617.55\..*smv$", readdir("../data/"))

@manipulate for i=slider(0:length(smvFilenames)-1, label="Iteration", value=length(smvFilenames)-1, continuous_update=true)
    smvSpectrum = SeaMass.SmvSpectrum("../data/" * smvFilenames[i+1], 1)
    seamassBinCounts = smiSpectrum.binCounts - smvSpectrum.residualBinCounts
    
    plot(smiSpectrum.binEdges, vcat(smiSpectrum.binCounts  ./ smiBinWidths, 0.0), line=:steppost, label="SMI input")
    plot!(smiSpectrum.binEdges, vcat(seamassBinCounts ./ smiBinWidths, 0.0), line=:steppost, label="seaMass prediction")
    plot!(xlabel = "m/z (Th)", ylabel = "ion density (counts per Th)")
    plot!(ylims = (0., 35000.))
    plot!(xlims = (684.2, 684.4), ylims = (0., 35000.))
    #plot!(xlims = (688.2, 688.4), ylims = (0., 3000.))
end

In [17]:
smiBinWidths = smiSpectrum.binEdges[2:end] - smiSpectrum.binEdges[1:end-1]

smoFilenames = filter!(r"^P02U_Swath_1_1D\.pos_617.55\..*smo$", readdir("../data/"))

@manipulate for i=slider(0:length(smoFilenames)-1, label="Iteration", value=length(smoFilenames)-1, continuous_update=true)
    smoSpectrum = SeaMass.SmoSpectrum("../data/" * smoFilenames[i+1], 1)
    sampledSmoSpectrum = SeaMass.MzmlSpectrum(smoSpectrum, 10000)
    
    plot(smiSpectrum.binEdges, vcat(smiSpectrum.binCounts  ./ smiBinWidths, 0.0), line=:steppost, label="SMI input")
    plot!(sampledSmoSpectrum.mzs, sampledSmoSpectrum.intensities, label="sampled reconstruction")
    plot!(xlabel = "m/z (Th)", ylabel = "ion density (counts per Th)")
    #plot!(ylims = (0., 35000.))
    plot!(xlims = (684.2, 684.4), ylims = (0., 35000.))
    #plot!(xlims = (688.2, 688.4), ylims = (0., 3000.))
end


Interact.Slider{Int64}(Signal{Int64}(286, nactions=1),"Iteration",286,0:286,"horizontal",true,"d",true)